In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set a random seed for reproducibility
np.random.seed(42)

# Create a dataset with 1000 samples and 5 feature columns
n_samples = 1000
n_features = 5

# Generate random feature data (values between 0 and 1)
features_data = np.random.rand(n_samples, n_features)

# Generate random target data (PressureLevel) - using values between 0 and 100
target_data = np.random.randint(0, 100, size=n_samples)

# Convert to a DataFrame for features and a Series for the target
columns = [f'feature_{i}' for i in range(1, n_features+1)]
df = pd.DataFrame(features_data, columns=columns)
df['PressureLevel'] = target_data

# Show the first few rows of the DataFrame
df.head()


,feature_1,feature_2,feature_3,feature_4,feature_5,PressureLevel
0,0.374540,0.950714,0.731994,0.598658,0.156019,50
1,0.155995,0.058084,0.866176,0.601115,0.708073,70
2,0.020584,0.969910,0.832443,0.212339,0.181825,95
3,0.183405,0.304242,0.524756,0.431945,0.291229,47
4,0.611853,0.139494,0.292145,0.366362,0.456070,18


In [2]:
# Split the dataset into features and target
features = df.drop(['PressureLevel'], axis=1)
target = df['PressureLevel']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Print the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (800, 5)
X_test shape: (200, 5)
y_train shape: (800,)
y_test shape: (200,)


In [3]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

y_train = y_train.ravel().reshape(-1, 1)
y_test = y_test.ravel().reshape(-1, 1)
y_scaler = StandardScaler().fit(y_train)

# Features
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# Target
y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

C:\Users\aryan\AppData\Local\Temp\ipykernel_13048\3483093910.py:5: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel().reshape(-1, 1)
C:\Users\aryan\AppData\Local\Temp\ipykernel_13048\3483093910.py:6: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test = y_test.ravel().reshape(-1, 1)


In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(10, activation='tanh', input_shape=(X_train.shape[1],)))
model.add(Dense(1, activation='linear'))


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

C:\Users\aryan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


The TerminateOnNan callback allows to stop the training when the loss function has diverged (by NaN value). 

This callback can be useful to avoid continuing to train the model if it has diverged or to stop the computation in case of non conforming data.


In [6]:
from tensorflow.keras import callbacks

TON = callbacks.TerminateOnNaN()

model.fit(X_train, y_train, 
          batch_size=32, 
          epochs=10,
          validation_split=0.2, 
          callbacks=[TON])

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.2455 - mean_absolute_error: 1.2365 - val_loss: 2.3201 - val_mean_absolute_error: 1.2446
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.0466 - mean_absolute_error: 1.1756 - val_loss: 2.0725 - val_mean_absolute_error: 1.1744
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8055 - mean_absolute_error: 1.1174 - val_loss: 1.8751 - val_mean_absolute_error: 1.1203
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5742 - mean_absolute_error: 1.0259 - val_loss: 1.7162 - val_mean_absolute_error: 1.0776
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5331 - mean_absolute_error: 1.0401 - val_loss: 1.5807 - val_mean_absolute_error: 1.0393
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3773 - mean_absolute_error: 0.9883 - val_loss: 1.4734 - val_mean_absolute_error: 1.0095
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2999 - mean_absolute_error: 0.9598 - val_loss: 1.383

LearningRateScheduler is a callback which allows you to vary the learning rate during training.


This callback takes as argument a function that updates the learning rate according to the epoch.


In [7]:
# If the current epoch number is divisible by 3 (epoch % 3 == 0), 
# the learning rate is reduced to 10% of its previous value (learning_rate * 0.1).

def decreasinglrUpdate(epoch, learning_rate):
    if epoch % 3 == 0:
        return learning_rate * 0.1
    else:
        return learning_rate


lrScheduler = callbacks.LearningRateScheduler(schedule=decreasinglrUpdate, verbose=1)

model.fit(X_train, y_train, 
          batch_size=32, 
          epochs=10,
          validation_split=0.2, 
          callbacks=[lrScheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.00010000000474974513.
Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1699 - mean_absolute_error: 0.9228 - val_loss: 1.1915 - val_mean_absolute_error: 0.9301 - learning_rate: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.00010000000474974513.
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1136 - mean_absolute_error: 0.9000 - val_loss: 1.1872 - val_mean_absolute_error: 0.9287 - learning_rate: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.00010000000474974513.
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0831 - mean_absolute_error: 0.8789 - val_loss: 1.1827 - val_mean_absolute_error: 0.9272 - learning_rate: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 1.0000000474974514e-05.
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0797 - mean_absolute_error: 0.8731 - val_loss: 1.1823 - val_mean_absolute_error: 0.9271 

EarlyStopping is a widely used callback. It allows you to control the evolution of the metrics by stopping the training when they no longer improve.


This callback is very useful to reduce the overfitting of a model.

- monitor = 'val_loss' (name of the metric to be controlled)

- patience = 3 (number of epochs to wait before interrupting the training process)

- mode = 'min' (allows to specify if the metric should increase or decrease: here, one should choose 'min' since the metric is a loss that we are trying to minimize)

- restore_best_weights = True (to restore the weights of the best epoch in the specified metric)


In [8]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         patience=3,
                                         mode='min',
                                         restore_best_weights=True)

model.fit(X_train, y_train, 
          batch_size=32, 
          epochs=20,
          validation_split=0.2, 
          callbacks=[early_stopping, lrScheduler])


Epoch 1: LearningRateScheduler setting learning rate to 1.000000082740371e-08.
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0146 - mean_absolute_error: 0.8545 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268 - learning_rate: 1.0000e-08

Epoch 2: LearningRateScheduler setting learning rate to 1.000000082740371e-08.
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1395 - mean_absolute_error: 0.9203 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268 - learning_rate: 1.0000e-08

Epoch 3: LearningRateScheduler setting learning rate to 1.000000082740371e-08.
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1184 - mean_absolute_error: 0.8994 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268 - learning_rate: 1.0000e-08

Epoch 4: LearningRateScheduler setting learning rate to 1.000000082740371e-09.
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1223 - mean_absolute_error: 0.9097 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268 - le

ModelCheckpoint callback 

It allows to save a model regularly during training, which can be useful during a long learning process.

- filepath = filepath (path to backup folder)
- monitor = 'val_loss' (name of the metric to be controlled)
- save_best_only = True (so that the best model is not overwritten)
- save_weights_only = False (so that the model does not save only the weights)
- mode = 'min' (allows to specify if the metric must increase or decrease: here we choose 'min' because the metric is a loss to minimize)
- save_freq = 'epoch' (so that the model is saved after each epoch)


In [17]:
import os
cwd = os.getcwd()
filepath = cwd + "/"

checkpoint = callbacks.ModelCheckpoint(filepath=filepath  + "model.keras", 
                                       monitor='val_loss',
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='min',
                                       save_freq='epoch')

model.fit(X_train, y_train, 
          batch_size=32, 
          epochs=5, 
          validation_split=0.2, 
          callbacks=[early_stopping, checkpoint])

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.1101 - mean_absolute_error: 0.8901 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1015 - mean_absolute_error: 0.8947 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1342 - mean_absolute_error: 0.9154 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268


ReduceLROnPlateau callback

If we want to change the learning rate according to the metric and not the epoch, we can use the ReduceLROnPlateau callback.

- monitor = 'val_loss' (name of the metric to be controlled)
- patience = 2 (number of epochs to wait before stopping the training)
- verbose = 2 (number of information that will be displayed during the learning process)
- mode = 'min' (allows to specify if the metric must increase or decrease: here we choose 'min' because the metric is a loss to minimize)


In [18]:
lr_plateau = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                         patience=2,
                                         verbose=2,
                                         mode='min')

model.fit(X_train, y_train, 
          batch_size=32, 
          epochs=50,
          validation_split=0.2, 
          callbacks=[early_stopping, lr_plateau])

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1193 - mean_absolute_error: 0.9073 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268 - learning_rate: 1.0000e-09
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0911 - mean_absolute_error: 0.8883 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268 - learning_rate: 1.0000e-09
Epoch 3/50
 1/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.4080 - mean_absolute_error: 1.0087
Epoch 3: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1563 - mean_absolute_error: 0.9089 - val_loss: 1.1812 - val_mean_absolute_error: 0.9268 - learning_rate: 1.0000e-09


In [20]:
class CustomCallBack(callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        print("For the batch {}, the train error is {:7.2f}.".format(batch, logs['loss']))

    def on_epoch_end(self, epoch, logs=None):
        print("Loss of validation for the epoch {} is {:7.2f} and the metric is {:7.2f}.".format(epoch, logs['val_loss'], logs['val_mean_absolute_error']))

    def on_test_end(self, logs=None):
        print("Validation completed.")


history = model.fit(X_train, y_train, 
                    batch_size=64, 
                    epochs=10,
                    validation_split=0.2, 
                    verbose=0,              #  To only take into account the messages displayed by the callback (and not by the fit method)
                    callbacks=[CustomCallBack()])

For the batch 0, the train error is    1.07.
For the batch 1, the train error is    1.10.
For the batch 2, the train error is    1.20.
For the batch 3, the train error is    1.12.
For the batch 4, the train error is    1.15.
For the batch 5, the train error is    1.12.
For the batch 6, the train error is    1.08.
For the batch 7, the train error is    1.08.
For the batch 8, the train error is    1.08.
For the batch 9, the train error is    1.11.
Validation completed.
Loss of validation for the epoch 0 is    1.18 and the metric is    0.93.
For the batch 0, the train error is    1.17.
For the batch 1, the train error is    1.18.
For the batch 2, the train error is    1.18.
For the batch 3, the train error is    1.18.
For the batch 4, the train error is    1.19.
For the batch 5, the train error is    1.16.
For the batch 6, the train error is    1.14.
For the batch 7, the train error is    1.10.
For the batch 8, the train error is    1.14.
For the batch 9, the train error is    1.11.
Valid